In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras 
import pandas as pd

In [2]:
orig_data = pd.read_csv("data_set/从11个特征预测学生的测试分数.csv")
X , Y = orig_data.iloc[:,0:10] , orig_data.iloc[:,-1] #切片
X = X.drop(columns=["school","classroom","student_id"],axis=1)#按列丢弃无关项
X = pd.get_dummies(X,columns=["school_setting","school_type","teaching_method","gender","lunch"],dtype=int)#独热编码 #orig_data.iloc[0:5,] X.iloc[0:5,]
X

,n_student,pretest,school_setting_Rural,school_setting_Suburban,school_setting_Urban,school_type_Non-public,school_type_Public,teaching_method_Experimental,teaching_method_Standard,gender_Female,gender_Male,lunch_Does not qualify,lunch_Qualifies for reduced/free lunch
0,20.0,62.0,0,0,1,1,0,0,1,1,0,1,0
1,20.0,66.0,0,0,1,1,0,0,1,1,0,1,0
2,20.0,64.0,0,0,1,1,0,0,1,0,1,1,0
3,20.0,61.0,0,0,1,1,0,0,1,1,0,1,0
4,20.0,64.0,0,0,1,1,0,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2128,30.0,39.0,0,0,1,0,1,0,1,1,0,1,0
2129,30.0,38.0,0,0,1,0,1,0,1,1,0,0,1
2130,30.0,45.0,0,0,1,0,1,0,1,1,0,0,1
2131,30.0,46.0,0,0,1,0,1,0,1,0,1,0,1


一层神经网络模型  
激活函数f(x)= x  
损失函数均方误差  
优化器Adam(默认比较优秀的优化器 结合了自适应梯度Adagrad和RMSProp防梯度急剧下降的优点)

In [3]:
model = keras.Sequential(
    [
        keras.layers.Dense(units=1,activation='linear',name = 'IN_PUT',input_shape=(13,)),
     ]
)#Sequential模型结构 多个网络层且无多余分支的堆叠 Dense就是常用的全连接层 运算就是output = activation(dot(input, kernel)+bias) units为该层的输出维度 而非输入维度  
model.compile(loss = keras.losses.MeanSquaredError(),optimizer = keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, amsgrad=False)) #adam（前期降低损失迅速，随迭代次数衰减学习率）学习率提高一点 
model.fit(X,Y,epochs=100,validation_split=0.2,validation_freq=10,batch_size=32) #整个数据一共遍历epoch遍 每一次拿batch_size个样本来算损失 所以一个epoch需要算epoch/batch_size遍 更新同样次数的参数
model.summary() 
w1,b1 = model.get_layer("IN_PUT").get_weights()
print("特征对应相关的系数w1如下:")
for i in range(len(X.columns)):
    print("{:<40}  {:>20}".format(str(X.columns[i]),str(w1[i])))
print("系数b1:{}".format(str(b1)))

Epoch 1/100
54/54 [==============================] - 0s 996us/step - loss: 1433.4374
Epoch 2/100
54/54 [==============================] - 0s 883us/step - loss: 124.9261
Epoch 3/100
54/54 [==============================] - 0s 834us/step - loss: 53.3984
Epoch 4/100
54/54 [==============================] - 0s 957us/step - loss: 48.3606
Epoch 5/100
54/54 [==============================] - 0s 994us/step - loss: 43.5534
Epoch 6/100
54/54 [==============================] - 0s 949us/step - loss: 38.9157
Epoch 7/100
54/54 [==============================] - 0s 1ms/step - loss: 34.6806
Epoch 8/100
54/54 [==============================] - 0s 1ms/step - loss: 30.9313
Epoch 9/100
54/54 [==============================] - 0s 834us/step - loss: 27.6490
Epoch 10/100
54/54 [==============================] - 0s 3ms/step - loss: 24.9015 - val_loss: 27.8869
Epoch 11/100
54/54 [==============================] - 0s 950us/step - loss: 22.6165
Epoch 12/100
54/54 [==============================] - 0s 959us/step 

换个优化器 随机梯度下降

In [4]:
model = keras.Sequential(
    [
        keras.layers.Dense(units=1,activation='linear',name = 'IN_PUT',input_shape=(13,)),
     ]
)
model.compile(loss = keras.losses.MeanSquaredError(),optimizer = keras.optimizers.SGD(learning_rate=0.01,clipvalue=0.5))#梯度裁剪 Gradient Clipping +-0.5
model.fit(X,Y,epochs=100,validation_split=0.2,validation_freq=10,batch_size=32) 
model.summary() 
w1,b1 = model.get_layer("IN_PUT").get_weights()
print("特征对应相关的系数w1如下:")
for i in range(len(X.columns)):
    print("{:<40}  {:>20}".format(str(X.columns[i]),str(w1[i])))
print("系数b1:{}".format(str(b1)))

Epoch 1/100
54/54 [==============================] - 0s 724us/step - loss: 4113.5874
Epoch 2/100
54/54 [==============================] - 0s 975us/step - loss: 1722.8843
Epoch 3/100
54/54 [==============================] - 0s 764us/step - loss: 388.4107
Epoch 4/100
54/54 [==============================] - 0s 686us/step - loss: 52.7168
Epoch 5/100
54/54 [==============================] - 0s 758us/step - loss: 35.8078
Epoch 6/100
54/54 [==============================] - 0s 837us/step - loss: 25.8905
Epoch 7/100
54/54 [==============================] - 0s 876us/step - loss: 19.8818
Epoch 8/100
54/54 [==============================] - 0s 896us/step - loss: 16.6951
Epoch 9/100
54/54 [==============================] - 0s 749us/step - loss: 15.3336
Epoch 10/100
54/54 [==============================] - 0s 2ms/step - loss: 14.4935 - val_loss: 12.9766
Epoch 11/100
54/54 [==============================] - 0s 914us/step - loss: 13.7103
Epoch 12/100
54/54 [==============================] - 0s 803us

自适应梯度 (对初始位置要求很严格 不然直接3000遍历还有1000损失 下降平缓均匀

In [5]:
model = keras.Sequential(
    [
        keras.layers.Dense(units=1,activation='linear',name = 'IN_PUT',input_shape=(13,)),
     ]
)
model.compile(loss = keras.losses.MeanSquaredError(),optimizer = keras.optimizers.Adagrad(learning_rate=0.01,epsilon=1e-6))
model.fit(X,Y,epochs=100,validation_split=0.2,validation_freq=10,batch_size=32) 
model.summary() 
w1,b1 = model.get_layer("IN_PUT").get_weights()
print("特征对应相关的系数w1如下:")
for i in range(len(X.columns)):
    print("{:<40}  {:>20}".format(str(X.columns[i]),str(w1[i])))
print("系数b1:{}".format(str(b1)))

Epoch 1/100
54/54 [==============================] - 0s 1ms/step - loss: 1789.4215
Epoch 2/100
54/54 [==============================] - 0s 1ms/step - loss: 1289.4225
Epoch 3/100
54/54 [==============================] - 0s 846us/step - loss: 1027.0938
Epoch 4/100
54/54 [==============================] - 0s 918us/step - loss: 845.3806
Epoch 5/100
54/54 [==============================] - 0s 736us/step - loss: 709.0980
Epoch 6/100
54/54 [==============================] - 0s 866us/step - loss: 602.5774
Epoch 7/100
54/54 [==============================] - 0s 853us/step - loss: 516.8130
Epoch 8/100
54/54 [==============================] - 0s 683us/step - loss: 446.4326
Epoch 9/100
54/54 [==============================] - 0s 861us/step - loss: 387.8655
Epoch 10/100
54/54 [==============================] - 0s 3ms/step - loss: 338.5983 - val_loss: 220.4021
Epoch 11/100
54/54 [==============================] - 0s 874us/step - loss: 296.8811
Epoch 12/100
54/54 [==============================] - 0s

结合梯度平方的指数移动平均数来调节学习率的变化。能够在不稳定（Non-Stationary）的目标函数情况下进行很好地收敛。递归神经网络时的一个良好选择？

In [6]:
model = keras.Sequential(
    [
        keras.layers.Dense(units=1,activation='linear',name = 'IN_PUT',input_shape=(13,)),
     ]
)
model.compile(loss = keras.losses.MeanSquaredError(),optimizer = keras.optimizers.RMSprop(learning_rate=0.01, rho=0.9, epsilon=1e-6)) #rho衰减系数 epsilon数值稳定性的小常数？
model.fit(X,Y,epochs=100,validation_split=0.2,validation_freq=10,batch_size=32) 
model.summary() 
w1,b1 = model.get_layer("IN_PUT").get_weights()
print("特征对应相关的系数w1如下:")
for i in range(len(X.columns)):
    print("{:<40}  {:>20}".format(str(X.columns[i]),str(w1[i])))
print("系数b1:{}".format(str(b1)))

Epoch 1/100
54/54 [==============================] - 0s 1ms/step - loss: 5193.2158
Epoch 2/100
54/54 [==============================] - 0s 965us/step - loss: 1063.4326
Epoch 3/100
54/54 [==============================] - 0s 1ms/step - loss: 107.0319
Epoch 4/100
54/54 [==============================] - 0s 1ms/step - loss: 45.8966
Epoch 5/100
54/54 [==============================] - 0s 1ms/step - loss: 22.6281
Epoch 6/100
54/54 [==============================] - 0s 1ms/step - loss: 16.3434
Epoch 7/100
54/54 [==============================] - 0s 1ms/step - loss: 14.3593
Epoch 8/100
54/54 [==============================] - 0s 1ms/step - loss: 13.2831
Epoch 9/100
54/54 [==============================] - 0s 1ms/step - loss: 12.5435
Epoch 10/100
54/54 [==============================] - 0s 3ms/step - loss: 12.2024 - val_loss: 11.7068
Epoch 11/100
54/54 [==============================] - 0s 1ms/step - loss: 11.9223
Epoch 12/100
54/54 [==============================] - 0s 1ms/step - loss: 11.909